In [2]:
import pandas as pd
import re

In [3]:
from bs4 import BeautifulSoup

with open('the2017report.html', 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'html.parser')

In [4]:
print(soup.prettify().split('#231F20;">')[5][:15])
print(len(soup.prettify().split('#231F20;">')[5].split('<p class="s1'))-1)


    CALIFORNIA
190


In [13]:
nums = len(soup.prettify().split('#231F20;">'))
nums

56

In [15]:
soup.prettify().split('#231F20;">')[nums-1][:15]

'\n    SWEDEN\n   '

In [20]:
# soup.prettify().split('#231F20;">')[0]

In [27]:
sn = []
for i in range(1,nums):
    state_text = soup.prettify().split('#231F20;">')[i][:15]
    sta = state_text
    num = len(soup.prettify().split('#231F20;">')[i].split('<p class="s1'))-1
    sn.append([sta, num])

In [38]:
df_state = pd.DataFrame(sn).T
df_state.columns = df_state.iloc[0]
df_state = df_state[1:]
df_state

,\n ALASKA\n,\n ALABAMA\n,\n ARKANSAS\n,\n ARIZONA\n,\n CALIFORNIA,\n COLORADO\n,\n CONNECTICU,\n DELAWARE\n,\n FLORIDA\n,\n GEORGIA\n,...,\n WYOMING\n,\n AUSTRALIA\n,\n CANADA\n,\n GERMANY\n,\n GREAT BRIT,\n NORTH IREL,\n MEXICO\n,\n PUERTO RIC,\n TAIWAN\n,\n SWEDEN\n
1,5,20,53,31,190,46,35,18,112,84,...,12,1,3,9,0,1,2,1,4,3


**loop**

In [27]:
res = []
for j in range(calilen):
    cl = soup.prettify().split('#231F20;">')[5].split('<p class="s1')[j].split('</p>\n')
    row = [extract_all_content_between_newlines(c) for c in cl]
    processed_values = [process_data(value) if value.startswith('<a') else value for value in row]
    flattened_list = flatten_list(processed_values)
    row_c1 = [remove_closing_angle_brackets(c1) for c1 in flattened_list]
    res.append(row_c1)

In [28]:
len(res)

190

In [29]:
df = pd.DataFrame(res)

In [30]:
# df

In [31]:
df.to_excel('aopa_cali.xlsx',index=False)

**trouble-shooting**